In [14]:
import pandas as pd
import matplotlib.pyplot as plt
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_train.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [15]:
x = df_train.drop('ACTION',axis=1)      #input features
y = df_train['ACTION']  


In [16]:
test_input = df_test.drop('id',axis=1)      #test input containing features
test_input.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,78766,72734,118079,118080,117878,117879,118177,19721,117880
1,40644,4378,117961,118327,118507,118863,122008,118398,118865
2,75443,2395,117961,118300,119488,118172,301534,249618,118175
3,43219,19986,117961,118225,118403,120773,136187,118960,120774
4,42093,50015,117961,118343,119598,118422,300136,118424,118425


In [17]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()
x, y = nm.fit_resample(x,y)

In [18]:
from sklearn.model_selection import train_test_split
#splitting in 20:80 for validation of data
x_train, x_valid, y_train, y_valid = train_test_split(x,y,test_size=0.2,random_state=1)

In [19]:
from catboost import CatBoostClassifier                 #using catboost because its fast and doesnt require pre-processing
from sklearn.metrics import f1_score

In [7]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()

In [8]:
x_train,y_train = oversample.fit_resample(x_train,y_train)

In [20]:
cat_features = list(range(x_train.columns.size))        #classified features on which our model will train

model = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric="F1",
    random_seed=1,
    verbose=100,
    cat_features = cat_features
)

model.fit(x_train,y_train,eval_set=(x_valid,y_valid),use_best_model=True)

Learning rate set to 0.041687
0:	learn: 0.8175773	test: 0.7975309	best: 0.7975309 (0)	total: 133ms	remaining: 2m 12s
100:	learn: 0.8855972	test: 0.8744828	best: 0.8744828 (100)	total: 6.75s	remaining: 1m
200:	learn: 0.9030207	test: 0.8814505	best: 0.8814505 (200)	total: 12.6s	remaining: 50.1s
300:	learn: 0.9144317	test: 0.8814505	best: 0.8833333 (243)	total: 18.4s	remaining: 42.8s
400:	learn: 0.9225848	test: 0.8845619	best: 0.8873435 (360)	total: 24.7s	remaining: 36.9s
500:	learn: 0.9337176	test: 0.8845619	best: 0.8873435 (360)	total: 31s	remaining: 30.9s
600:	learn: 0.9434811	test: 0.8861111	best: 0.8873435 (360)	total: 36.9s	remaining: 24.5s
700:	learn: 0.9474359	test: 0.8888889	best: 0.8888889 (672)	total: 43.9s	remaining: 18.7s
800:	learn: 0.9512821	test: 0.8888889	best: 0.8891967 (751)	total: 50s	remaining: 12.4s
900:	learn: 0.9576108	test: 0.8891967	best: 0.8891967 (751)	total: 56.3s	remaining: 6.18s
999:	learn: 0.9627728	test: 0.8876560	best: 0.8891967 (751)	total: 1m 2s	remaini

In [22]:
predictions = model.predict(test_input)

In [23]:
result = pd.DataFrame({'Id':df_test['id'],'Action':predictions})
result.to_csv('Submission15.csv',index=False)
#BY CATBOOST

In [ ]:
#BY ADABOOST
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(learning_rate=10**-3,n_estimators=100,random_state=1)
clf.fit(x_train,y_train)

AdaBoostClassifier(learning_rate=0.001, n_estimators=100, random_state=1)

In [ ]:
clf.predict(test_input)

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [ ]:
clf.score(x_valid,y_valid)

0.7509917607567898

In [ ]:
result = pd.DataFrame({'Id':df_test['id'],'Action':clf.predict(test_input)})
result.to_csv('Submission11.csv',index=False)

In [ ]:
!pip install xgboost


In [25]:
import xgboost as xgb
model_xgb = xgb.XGBClassifier(learning_rate=1,n_estimators=100,eval_metric='auc')
eval_set = [(x_valid,y_valid)]
model_xgb.fit(x_train,y_train,eval_set=eval_set)

[0]	validation_0-auc:0.90974
[1]	validation_0-auc:0.91451
[2]	validation_0-auc:0.92746
[3]	validation_0-auc:0.93106
[4]	validation_0-auc:0.93254
[5]	validation_0-auc:0.93416
[6]	validation_0-auc:0.93681
[7]	validation_0-auc:0.93855
[8]	validation_0-auc:0.94712
[9]	validation_0-auc:0.94954
[10]	validation_0-auc:0.94979
[11]	validation_0-auc:0.95060
[12]	validation_0-auc:0.95144
[13]	validation_0-auc:0.95018
[14]	validation_0-auc:0.94887
[15]	validation_0-auc:0.94870
[16]	validation_0-auc:0.94759
[17]	validation_0-auc:0.94766
[18]	validation_0-auc:0.94827
[19]	validation_0-auc:0.94884
[20]	validation_0-auc:0.95071
[21]	validation_0-auc:0.95001
[22]	validation_0-auc:0.95077
[23]	validation_0-auc:0.95101
[24]	validation_0-auc:0.95154
[25]	validation_0-auc:0.94929
[26]	validation_0-auc:0.94992
[27]	validation_0-auc:0.94999
[28]	validation_0-auc:0.95083
[29]	validation_0-auc:0.95062
[30]	validation_0-auc:0.95108
[31]	validation_0-auc:0.95097
[32]	validation_0-auc:0.95165
[33]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=1, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [26]:
result = pd.DataFrame({'Id':df_test['id'],'Action':model_xgb.predict(test_input)})
result.to_csv('Submission17.csv',index=False)

In [27]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(weights='distance')
neigh.fit(x_train,y_train)

KNeighborsClassifier(weights='distance')

In [28]:
neigh.score(x_valid,y_valid)

0.8208168642951251

In [29]:
result = pd.DataFrame({'Id':df_test['id'],'Action':neigh.predict(test_input)})
result.to_csv('Submission18.csv',index=False)